In [1]:
import random
import math
import numba
from copy import deepcopy

from chainreaction.game import *
from chainreaction.positions import *
from chainreaction.board import *
from chainreaction.mcts import *
from chainreaction.four_players import *

In [6]:
w, h = 3, 3
state = Board(w, h)
state.place_atom(0, 0, FourPlayers.P1.value)
state.place_atom(0, 2, FourPlayers.P2.value)
state.place_atom(2, 0, FourPlayers.P34.value)
state.place_atom(2, 2, FourPlayers.P34.value)

In [7]:
def mcts_step(state):
    start_player = FourPlayers.P1
    # make sure we are calling the right type of mcts node
    # maybe handle it in base mcts node class
    root = MCTSNormalNode(state, parent=None, parent_action=None, player=start_player)
    return None, root.best_action().parent_action

In [8]:
player = FourPlayers.P1
temp_player = player
while True:
    _, move = minimax_step(state.get_copy(), deepcopy(player), 0)
    moves_count += 1
    print(f'{player} placing on {move}')
    state, player, utilities, terminal = game_step(state, player, move)
    print("New State:")
    print(format_board(state))
    if terminal and moves_count>4:
        max_value = max(utilities)
        won_players = [i+1 for i, j in enumerate(utilities) if j == max_value]
        if len(won_players)==1:
            print("Game Won by Player " + str(won_players[0]))
        else:
            won_players_ints = [str(i) for i in won_players]
            print("Game Won by Players " + (", ".join(won_players_ints)))
        break
    temp_player = player
    print('=='*20)

(1, 2)